### Welcome to the LangChain demo!!

In [ ]:
# Load the environment variables
import dotenv, os
dotenv.load_dotenv()
print(os.environ['OPENAI_API_KEY'])

### Basic Chat Model

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")
model.invoke("Hello AI overlord!!")

### Message Types
Explicit construction of messages

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("I'm excited to get started with the problem-first course!"),
]
model.invoke(messages)

### Prompt Templates

In [ ]:
# Messages
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

system_message = "You are a wise and mystical AI fortune teller. Your predictions are funny, slightly exaggerated, but insightful. Keep it to 1-2 sentences"
user_message = "Please entertain the user with their fortune request: {question}"
prompt_template = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Construct the prompt from the template:
question = "What is the next trillion dollar idea?"
prompt = prompt_template.invoke({"question": question})

# Check what's in the prompt
print(prompt)

In [ ]:
# Send the prompt to the model and get the result
result = model.invoke(prompt)
print(result)

In [ ]:
type(result)

In [ ]:
## Output parser to convert it to a text:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

parser_result = output_parser.invoke(result)

In [ ]:
type(parser_result)

### Chaining
Let's chain the output of the first model to the next one.

In [ ]:
# Remember: Use the template here directly in the chain instead of 'prompt'
first_chain = prompt_template | model | output_parser
first_chain.invoke({"question": question})

In [ ]:
# Create 2nd prompt template:
system_message = "You are a stand-up comedian who tells hilarious jokes in a casual, witty style using AI terminology. Keep it short"
user_message = "Tell a joke about this fortune telling: {fortune}."
prompt_template_2 = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Chain the 1st and 2nd prompts
new_chain = prompt_template | model | output_parser | (lambda x: {"fortune": x}) | prompt_template_2 | model | output_parser
new_chain.invoke({"question": question})

In [ ]:
# The above is equivalent to running:
first_chain = prompt_template | model | output_parser
first_result = first_chain.invoke({"question": question})

second_chain = prompt_template_2 | model | output_parser
final_result = second_chain.invoke({"fortune": first_result})
print(final_result)